# Multi-table Datasets - ENRON Archive

## 1. Data import

Connect to the file 'assets/datasets/enron.db' using one of these methods:

- sqlite3 python package
- pandas.read_sql
- SQLite Manager Firefox extension

Take a look at the database and query the master table. How many Tables are there in the db?

> Answer:
There are 3 tables:
- MessageBase
- RecipientBase
- EmployeeBase

In [2]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('../../assets/datasets/enron.db')
c = conn.cursor()
c.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()

[(u'MessageBase',), (u'RecipientBase',), (u'EmployeeBase',)]

Query the `sqlite_master` table to retrieve the schema of the `EmployeeBase` table.

1. What fields are there?
1. What's the type of each of them?

In [46]:
db_data = c.execute("SELECT * FROM sqlite_master WHERE name ='EmployeeBase'")
labels = db_data.fetchall()[0][4].split('\n')

for col, i in enumerate(labels[1:8]):
    print "column", col , i

column 0                   [eid] INTEGER,
column 1   [name] TEXT,
column 2   [department] TEXT,
column 3   [longdepartment] TEXT,
column 4   [title] TEXT,
column 5   [gender] TEXT,
column 6   [seniority] TEXT


1. Print the first 5 rows of EmployeeBase table
1. Print the first 5 rows of MessageBase table
1. Print the first 5 rows of RecipientBase table

**Hint**  use `SELECT` and `LIMIT`.

In [54]:
results = c.execute('SELECT * FROM EmployeeBase limit 5 ')
fetched = results.fetchall()
for x in fetched:
    print x
print "\n"

results = c.execute('SELECT * FROM MessageBase limit 5 ')
fetched = results.fetchall()
for x in fetched:
    print x
print "\n"

results = c.execute('SELECT * FROM RecipientBase limit 5 ')
fetched = results.fetchall()
for x in fetched:
    print x
print "\n"

(1, u'John Arnold', u'Forestry', u'ENA Gas Financial', u'VP Trading', u'Male', u'Senior')
(2, u'Harry Arora', u'Forestry', u'ENA East Power', u'VP Trading', u'Male', u'Senior')
(3, u'Robert Badeer', u'Forestry', u'ENA West Power', u'Mgr Trading', u'Male', u'Junior')
(4, u'Susan Bailey', u'Legal', u'ENA Legal', u'Specialist Legal', u'Female', u'Junior')
(5, u'Eric Bass', u'Forestry', u'ENA Gas Texas', u'Trader', u'Male', u'Junior')


(1, u'taylor-m/sent/11', 910930020, u'Cd$ CME letter', 138)
(2, u'taylor-m/sent/17', 911459940, u'Indemnification', 138)
(3, u'taylor-m/sent/18', 911463840, u'Re: Indemnification', 138)
(4, u'taylor-m/sent/23', 911874180, u'Re: Coral Energy, L.P.', 138)
(5, u'taylor-m/sent/27', 912396120, u'Bankruptcy Code revisions', 138)


(1, 1, 59)
(2, 1, 15)
(3, 1, 15)
(4, 1, 109)
(4, 2, 49)




Import each of the 3 tables to a Pandas Dataframes

In [55]:
employee_table = pd.read_sql("SELECT * FROM EmployeeBase", conn)
message_table = pd.read_sql("SELECT * FROM MessageBase", conn)
recipient_table = pd.read_sql("SELECT * FROM RecipientBase", conn)

## 2. Data Exploration

Use the 3 dataframes to answer the following questions:

1. How many employees are there in the company?
- How many messages are there in the database?
- Convert the timestamp column in the messages. When was the oldest message sent? And the newest?
- Some messages are sent to more than one recipient. Group the messages by message_id and count the number of recepients. Then look at the distribution of recepient numbers.
    - How many messages have only one recepient?
    - How many messages have >= 5 recepients?
    - What's the highest number of recepients?
    - Who sent the message with the highest number of recepients?
- Plot the distribution of recepient numbers using Bokeh.

In [58]:
# 1. How many employees are there in the company?
print len(employee_table)

156


In [61]:
# 2. How many messages are there in the database?
print len(message_table)

21635


In [68]:
# 3. Convert the timestamp column in the messages. When was the oldest message sent? And the newest?
message_table['unix_time'] = pd.to_datetime(message_table['unix_time'],unit='s')
print 'Oldest Message Sent: ', min(message_table['unix_time'])
print 'Newest Message Sent: ', max(message_table['unix_time'])

Oldest Message Sent:  1998-11-13 04:07:00
Newest Message Sent:  2002-06-21 13:37:34


In [179]:
recipient_table.head()

,mid,rno,to_eid
0,1,1,59
1,2,1,15
2,3,1,15
3,4,1,109
4,4,2,49


In [189]:
message_table.head()

,mid,filename,unix_time,subject,from_eid
0,1,taylor-m/sent/11,1998-11-13 04:07:00,Cd$ CME letter,138
1,2,taylor-m/sent/17,1998-11-19 07:19:00,Indemnification,138
2,3,taylor-m/sent/18,1998-11-19 08:24:00,Re: Indemnification,138
3,4,taylor-m/sent/23,1998-11-24 02:23:00,"Re: Coral Energy, L.P.",138
4,5,taylor-m/sent/27,1998-11-30 03:22:00,Bankruptcy Code revisions,138


In [199]:
employee_table[['eid','name']].head()

,eid,name
0,1,John Arnold
1,2,Harry Arora
2,3,Robert Badeer
3,4,Susan Bailey
4,5,Eric Bass


In [272]:
#5. Some messages are sent to more than one recipient. Group the messages by message_id and count the number of recepients. 
#Then look at the distribution of recepient numbers.
message_sent = pd.merge(message_table, recipient_table, on='mid', how = 'inner')
message_count = pd.pivot_table(message_sent, index = ['mid'], values = ['rno'], aggfunc = len)

In [273]:
print 'Messages with 1 recepient: ', len(message_count[message_count['rno']==1])
print 'Messages with 5 or more recepient: ', len(message_count[message_count['rno']>=5])
print 'Message with most recepient: ', max(message_count['rno'])


Messages with 1 recepient:  14985
Messages with 5 or more recepient:  1380
Message with most recepient:  57


In [ ]:
mx = max(message_count['rno'])
message_count[message_count['rno']==mx].index.values

In [284]:
message_table[message_table['mid']==12116]

,mid,filename,unix_time,subject,from_eid
12115,12116,baughman-d/all_documents/398,2001-05-22 05:53:00,,67


In [279]:
employee_table[employee_table['eid']==67]

,eid,name,department,longdepartment,title,gender,seniority
66,67,John J. Lavorato,Other,EWS,ENA President & CEO,Male,Senior


Rescale to investigate the tail of the curve

In [286]:
from bokeh.charts import Histogram, show, output_notebook
from bokeh.io import output_notebook

x = Histogram(message_sent, values = 'rno')
output_notebook()
show(x)

Loading BokehJS ...

## 3. Data Merging

Use the pandas merge function to combine the information in the 3 dataframes to answer the following questions:

1. Are there more Men or Women employees?
- How is gender distributed across departments?
- Who is sending more emails? Men or Women?
- What's the average number of emails sent by each gender?
- Are there more Juniors or Seniors?
- Who is sending more emails? Juniors or Seniors?
- Which department is sending more emails? How does that relate with the number of employees in the department?
- Who are the top 3 senders of emails? (people who sent out the most emails)

In [287]:
# Q1 There are more men than women
employee_table.groupby('gender').count()

,eid,name,department,longdepartment,title,seniority
gender,,,,,,
Female,43,43,43,43,43,43
Male,113,113,113,113,113,113


In [288]:
# Q2 Distribution of males and females across departments
pd.pivot_table(employee_table[['department','gender']], index = ['department'], columns = ['gender'], aggfunc = len)

gender,Female,Male
department,,
Forestry,10,50
Legal,13,12
Other,20,51


In [299]:
# Q3 Who is sending more emails? Men or Women?
sender = pd.merge(message_table, employee_table, left_on = 'from_eid', right_on = 'eid', how = 'inner')
print 'male:', len(sender[sender['gender'] == 'Male'])
print 'female:', len(sender[sender['gender'] == 'Female'])

male: 12841
female: 8794


In [302]:
# Q4 Who is sending more emails? Men or Women?
a = pd.pivot_table(sender, index=['name'], values=['eid'], columns=['gender'], aggfunc=len)
print "Average messangs sent by women: ", np.mean(a[('eid', 'Female')])
print "Average messages sent by men: ", np.mean(a[('eid', 'Male')])

Average messangs sent by women:  214.487804878
Average messages sent by men:  115.684684685


In [304]:
# Q5 There are more Juniors than Seniors
employee_table.groupby('seniority').count()

,eid,name,department,longdepartment,title,gender
seniority,,,,,,
Junior,82,82,82,82,82,82
Senior,74,74,74,74,74,74


In [310]:
# Q6 Seniors send out more messages than juniors
print 'Junior: ', len(sender[sender['seniority']=='Junior'])
print 'Senior: ', len(sender[sender['seniority']=='Senior'])

Junior:  9196
Senior:  12439


In [312]:
# Q7 'ENA Legal' sents out the largest number of messages even though it's only the 4th largest department
print "Department by number of emails: \n", sender.groupby('longdepartment').count()['mid'].sort_values(ascending=False)[0:1], "\n" 
print employee_table.groupby('longdepartment').count()[('eid')].sort_values()

Department by number of emails: 
longdepartment
ENA Legal    6536
Name: mid, dtype: int64 

longdepartment
EES                                   1
NA                                    3
Regulatory and Government Affairs     4
ENA Gas Financial                     5
ENA Gas Texas                         5
Enron                                 5
Energy Operations                     8
EWS                                   9
ENA Gas Central                      10
ENA West Power                       10
ENA West Power Real Time             11
ENA Gas East                         12
ENA Legal                            17
ENA East Power                       18
ENA Gas West                         19
ETS                                  19
Name: eid, dtype: int64


In [314]:
# Q8 Top 3 senders
sender.groupby('name').count()['mid'].sort_values(ascending=False)[0:3]

name
Jeff Dasovich      1597
Tana Jones         1379
Sara Shackleton    1142
Name: mid, dtype: int64

Answer the following questions regarding received messages:

- Who is receiving more emails? Men or Women?
- Who is receiving more emails? Juniors or Seniors?
- Which department is receiving more emails? How does that relate with the number of employees in the department?
- Who are the top 5 receivers of emails? (people who received the most emails)

In [328]:
recipient = recipient_table.merge(employee_table, how = 'inner', left_on = 'to_eid', right_on = 'eid')

print 'Emails received by gender', recipient.groupby('gender')['mid'].nunique(), '\n'
print 'Emails received by seniority', recipient.groupby('seniority')['mid'].nunique(), '\n'
print 'Emails by department', pd.pivot_table(recipient, values = ['mid','eid'], index = ['department'], aggfunc = {'mid': len, 'eid': lambda x: len(x.unique())}), '\n'
print 'Top email recipient', recipient['name'].value_counts()[0:5]

Emails received by gender gender
Female     9617
Male      15446
Name: mid, dtype: int64 

Emails received by seniority seniority
Junior     9966
Senior    14556
Name: mid, dtype: int64 

Emails by department             eid    mid
department            
Forestry     60   8424
Legal        25  16311
Other        69  13653 

Top email recipient James D. Steffes    1797
Richard Shapiro     1730
Mark E. Taylor      1477
Steven J. Kean      1290
Sara Shackleton     1173
Name: name, dtype: int64


Which employees sent the most 'mass' emails?

In [330]:
spammer = recipient.groupby('mid').count().sort_values('rno', ascending = False)[0:1].index.values[0]
sender[sender['mid']==spammer]['name']

12238    John J. Lavorato
Name: name, dtype: object

Keep exploring the dataset, which other questions would you ask?

Work in pairs. Give each other a challenge and try to solve it.